In [275]:
import pandas as pd
from arcgis.features import GeoAccessor
from pathlib import Path
import os
import arcpy

In [107]:
data = Path(os.path.abspath('../../data'))
interim_dir = data/'interim'
interim_gdb = interim_dir/'interim.gdb'
raw_dir = data/'raw'
raw_gdb = raw_dir/'raw.gdb'

real_estate_fc = raw_gdb/'real_estate_hex'
real_estate_id_fld = 'GRID_ID'

In [136]:
real_estate_df = GeoAccessor.from_featureclass(str(real_estate_fc)).drop('OBJECTID', axis=1)
real_estate_df.head()

,GRID_ID,road_reachable,SHAPE
0,24,0,"{""rings"": [[[-122.11125931899994, 46.740620218..."
1,26,0,"{""rings"": [[[-122.04954545899994, 46.740620218..."
2,28,1,"{""rings"": [[[-121.98783159799996, 46.740620218..."
3,30,1,"{""rings"": [[[-121.92611773799996, 46.740620218..."
4,31,1,"{""rings"": [[[-121.89526080699994, 46.728410000..."


In [375]:
affected_df = pd.concat([pd.read_csv(f, index_col=0) for f in interim_dir.glob('ace_affected*.csv')]).drop_duplicates(['origin_id', 'new_destination_id'])
affected_df.head()

,delta_01,destination_id_01,destination_id_02,destination_id_03,destination_id_04,destination_id_05,destination_id_06,new_destination_id,origin_id,predict_new_01,trip_count_01,trip_count_02,trip_count_03,trip_count_04,trip_count_05,trip_count_06
34,2.146481,1.0,371889957.0,216082099.0,721714069.0,421027779.0,677129595.0,28,5.305307e+11,2.146481,0.0,1.0,0.0,0.0,0.0,0.0
34,2.146481,1.0,371889957.0,216082099.0,721714069.0,421027779.0,677129595.0,30,5.305307e+11,2.146481,0.0,1.0,0.0,0.0,0.0,0.0
34,2.146481,1.0,371889957.0,216082099.0,721714069.0,421027779.0,677129595.0,31,5.305307e+11,2.146481,0.0,1.0,0.0,0.0,0.0,0.0
34,2.146481,1.0,371889957.0,216082099.0,721714069.0,421027779.0,677129595.0,32,5.305307e+11,2.146481,0.0,1.0,0.0,0.0,0.0,0.0
34,2.146481,371889957.0,216082099.0,1.0,721714069.0,421027779.0,677129595.0,33,5.305307e+11,2.146481,0.0,1.0,0.0,0.0,0.0,0.0


In [394]:
import warnings
warnings.filterwarnings('ignore')

affected_df = pd.concat([pd.read_csv(f, index_col=0) for f in interim_dir.glob('ace_affected*.csv')]).drop_duplicates(['origin_id', 'new_destination_id'])

delta_srs = affected_df.groupby('new_destination_id').sum()['delta_01']
print(f'successfully completed - {len(delta_srs):,} / {len(real_estate_df[real_estate_df.road_reachable == 1].index):,}')

real_estate_df = GeoAccessor.from_featureclass(str(real_estate_fc)).drop('OBJECTID', axis=1)
real_estate_df.GRID_ID = real_estate_df.GRID_ID.astype('int64')
real_estate_delta_df = real_estate_df.join(delta_srs, on='GRID_ID').reset_index(drop=True)

webmap = real_estate_delta_df[real_estate_delta_df['delta_01'].notna()].spatial.plot()
webmap.basemap ='dark-gray-vector'
webmap

successfully completed - 716 / 1,906


MapView(layout=Layout(height='400px', width='100%'))

In [395]:
delta_srs = real_estate_delta_df.set_index('GRID_ID')['delta_01']
with arcpy.da.UpdateCursor(str(interim_gdb/'real_estate_potential'), ['GRID_ID', 'delta_01']) as update_cur:
    for row in update_cur:
        row[1] = delta_srs.loc[int(row[0])]
        update_cur.updateRow(row)

In [ ]:
real_estate_delta_df.spatial.to_featureclass(str(interim_gdb/'real_estate_potential'))